In [ ]:
import cv2
import numpy as np

def process_frame(frame):
    blue, _, _ = cv2.split(frame)
    ret2, binary = cv2.threshold(blue, 220, 255, 0)
    gaussian = cv2.GaussianBlur(binary, (5, 5), 0)

    draw_img = gaussian.copy()
    whole_h, _ = binary.shape[:2]

    _, contours, _ = cv2.findContours(image=draw_img, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, key=lambda c: cv2.contourArea(c), reverse=True)[:5]

    width_array = []
    height_array = []
    point_array = []

    for cont in contours:
        x, y, w, h = cv2.boundingRect(cont)
        try:
            if h / w >= 2 and h / whole_h > 0.1 and h > w:
                width_array.append(w)
                height_array.append(h)
                point_array.append([x, y])
        except ZeroDivisionError:
            continue

    point_near = [0, 0]
    min_value = 10000

    for i in range(len(width_array) - 1):
        for j in range(i + 1, len(width_array)):
            value = abs(width_array[i] * height_array[i] - width_array[j] * height_array[j])
            if value < min_value:
                min_value = value
                point_near[0] = i
                point_near[1] = j

    try:
        rectangle1 = point_array[point_near[0]]
        rectangle2 = point_array[point_near[1]]
        point1 = [rectangle1[0] + width_array[point_near[0]] / 2, rectangle1[1]]
        point2 = [rectangle1[0] + width_array[point_near[0]] / 2, rectangle1[1] + height_array[point_near[0]]]
        point3 = [rectangle2[0] + width_array[point_near[1]] / 2, rectangle2[1]]
        point4 = [rectangle2[0] + width_array[point_near[1]] / 2, rectangle2[1] + height_array[point_near[1]]]

        x = np.array([point1, point2, point4, point3], np.int32)
        box = x.reshape((-1, 1, 2)).astype(np.int32)
        cv2.polylines(frame, [box], True, (0, 255, 0), 2)

    except IndexError:
        pass

    return frame

# Open a connection to the camera (use 0 for the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    if not ret:
        print("Failed to capture frame")
        break

    # Process the frame
    processed_frame = process_frame(frame)

    # Display the processed frame
    cv2.imshow('Processed Frame', processed_frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()
